# Imports

In [107]:
import os
import pandas as pd

#python 3.11.9

## Notebook Overview

This notebook is designed to clean and preprocess a dataset of the top 500 supercomputers. The main tasks performed in this notebook include:

1. **Importing Necessary Libraries**:
    - The notebook imports essential libraries such as `pandas` and `os` for data manipulation and file operations.

2. **Defining Column Groups**:
    - `COLUMN_GROUPS` and `COLUMN_MAPPING` dictionaries are defined to group similar columns together. This helps in merging columns with similar data but different names.

3. **Unit Conversions**:
    - The `UNIT_CONVERSIONS` dictionary is used to convert units of specific columns to maintain consistency across the dataset.

4. **Data Cleaning Functions**:
    - `combine_columns(df)`: Merges similar columns into a single column, keeping the first non-null value.
    - `clean_csv(input_file, output_file, min_valid_count=2500, debug=True)`: Reads, cleans, and saves a CSV file. It performs unit conversions, combines similar columns, and filters out columns with less than a specified number of non-null values.

5. **Loading and Cleaning Data**:
    - The notebook reads the input CSV file, applies the cleaning functions, and saves the cleaned data to an output CSV file.

6. **Counting NaN Values**:
    - `count_nans(df)`: Returns the total number of NaN values in the DataFrame.
    - The notebook counts the number of NaN values before and after cleaning to evaluate the effectiveness of the cleaning process.

7. **Summary of Variables**:
    - Various variables such as `COLUMN_GROUPS`, `COLUMN_MAPPING`, `UNIT_CONVERSIONS`, and dataframes like `df_original`, `df_cleaned`, and `df_merged` are defined and used throughout the notebook.

8. **Example Usage**:
    - The notebook provides example usage of the cleaning functions with specific input and output file paths.

Overall, this notebook provides a comprehensive workflow for cleaning and preprocessing a dataset of supercomputers, ensuring consistency and reducing the number of missing values.


In [108]:
# Define similar column groups
COLUMN_GROUPS = {
    "Processors": ["Processors", "Cores", "Total Cores"], #completed and verified my hand
    "RMax (Flop/s)": ["RMax", "Rmax", "Rmax [TFlop/s]"], #completed and verified my hand
    "RPeak (Flop/s)": ["RPeak", "Rpeak", "Rpeak [TFlop/s]"],
    "Power (W)": ["Power", "Power (kW)"],
    "Accelerator": ["Accelerator", "Accelerator/Co-Processor"],
    "Accelerator Cores": ["Accelerator Cores", "Accelerator/Co-Processor Cores"]
}

def combine_columns(df):
    """Merges similar columns into a single column, keeping the first non-null value."""
    for new_col, old_cols in COLUMN_GROUPS.items():
        found_cols = [col for col in old_cols if col in df.columns]
        
        if len(found_cols) > 1:
            df[new_col] = df[found_cols].bfill(axis=1).iloc[:, 0]  # Take first non-null value
            df.drop(columns=[col for col in found_cols if col != new_col], inplace=True)  # Drop duplicates

    return df

def clean_csv(input_file, output_file):
    """Reads, cleans, and saves a CSV file."""
    
    df = pd.read_csv(input_file)
    df["Rmax [TFlop/s]"] = df["Rmax [TFlop/s]"] * 1e12
    df["Rpeak [TFlop/s]"] = df["Rpeak [TFlop/s]"] * 1e12
    df["Power (kW)"] = df["Power (kW)"] * 1e3
    
    df = combine_columns(df)
    columns_to_keep = [col for col in df.columns if df[col].notna().sum() >= 1000]
    df = df[columns_to_keep]

    df.to_csv(output_file, index=False)
    print(f"Cleaned CSV saved to {output_file}")

# Example usage
current_dir = os.getcwd()
input_csv = os.path.join(current_dir, "merged_top500_nans.csv")  # Change to your actual file
output_csv = os.path.join(current_dir, "merged_top500_clean.csv")

clean_csv(input_csv, output_csv)


C:\Users\Owner\AppData\Local\Temp\ipykernel_13000\433935421.py:25: DtypeWarning: Columns (29,30,37,39,41,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_file)


Cleaned CSV saved to c:\VSCode\360\top500_data\merged_top500_clean.csv


In [109]:
import pandas as pd
import os

# Define similar column groups
COLUMN_GROUPS = {
    "Processors": ["Processors", "Cores", "Total Cores"],
    "RMax (Flop/s)": ["RMax", "Rmax", "Rmax [TFlop/s]"],
    "RPeak (Flop/s)": ["RPeak", "Rpeak", "Rpeak [TFlop/s]"],
    "Power (W)": ["Power", "Power (kW)"],
    "Accelerator": ["Accelerator", "Accelerator/Co-Processor"],
    "Accelerator Cores": ["Accelerator Cores", "Accelerator/Co-Processor Cores"]
}

def combine_columns(df):
    """Merges similar columns into a single column, keeping the first non-null value."""
    for new_col, old_cols in COLUMN_GROUPS.items():
        found_cols = [col for col in old_cols if col in df.columns]
        
        if len(found_cols) > 1:
            # Create a new column with merged values
            df[new_col] = df[found_cols].bfill(axis=1).iloc[:, 0]
            # Drop original columns only if they're different from the new column name
            df.drop(columns=[col for col in found_cols if col != new_col], inplace=True)
        elif len(found_cols) == 1 and found_cols[0] != new_col:
            # If there's only one column found and it's different from the new name
            df[new_col] = df[found_cols[0]]
            df.drop(columns=found_cols[0], inplace=True)

    return df

def clean_csv(input_file, output_file, min_valid_count=2500, debug=True):
    """Reads, cleans, and saves a CSV file."""
    
    print(f"Reading file: {input_file}")
    df = pd.read_csv(input_file, low_memory=False)
    
    # Convert units if columns exist
    if "Rmax [TFlop/s]" in df.columns:
        df["Rmax [TFlop/s]"] = df["Rmax [TFlop/s]"] * 1e12
    if "Rpeak [TFlop/s]" in df.columns:
        df["Rpeak [TFlop/s]"] = df["Rpeak [TFlop/s]"] * 1e12
    if "Power (kW)" in df.columns:
        df["Power (kW)"] = df["Power (kW)"] * 1e3
    
    print(f"Original dataframe shape: {df.shape}")
    
    # Combine similar columns
    df = combine_columns(df)
    print(f"After combining columns, shape: {df.shape}")
    
    # Print non-null counts for debugging
    if debug:
        non_null_counts = df.count()
        print("\nNon-null counts for each column:")
        for col in df.columns:
            print(f"  {col}: {non_null_counts[col]}")
    
    # Create list of columns to keep
    columns_to_keep = []
    for col in df.columns:
        count = df[col].count()  # .count() excludes NaN values
        if count >= min_valid_count:
            columns_to_keep.append(col)
        elif debug:
            print(f"Dropping column '{col}' with only {count} non-null values")
    
    # Filter columns
    df_filtered = df[columns_to_keep]
    
    print(f"\nColumns kept: {len(columns_to_keep)} out of {len(df.columns)}")
    print(f"Final dataframe shape: {df_filtered.shape}")
    
    # Save the filtered dataframe
    df_filtered.to_csv(output_file, index=False)
    print(f"Cleaned CSV saved to {output_file}")
    
    return df_filtered

# Example usage
current_dir = os.getcwd()
input_csv = os.path.join(current_dir, "merged_top500_nans.csv")
output_csv = os.path.join(current_dir, "merged_top500_clean.csv")

# Run with debug info
clean_csv(input_csv, output_csv, min_valid_count=1000, debug=True)

Reading file: c:\VSCode\360\top500_data\merged_top500_nans.csv
Original dataframe shape: (32000, 59)
After combining columns, shape: (32000, 50)

Non-null counts for each column:
  Rank: 32000
  Site: 32000
  Manufacturer: 32000
  Computer: 32000
  Country: 32000
  Year: 32000
  Processors: 32000
  Nmax: 31558
  Nhalf: 24172
  Processor Family: 18500
  Processor: 32000
  Proc. Frequency: 18500
  System Family: 32000
  Operating System: 31915
  Architecture: 32000
  Segment: 32000
  Application Area: 18500
  Interconnect Family: 27426
  Interconnect: 27717
  Region: 27000
  Continent: 32000
  Publication Date: 32000
  System Model: 17000
  Measured Size: 500
  Processor Cores: 1500
  Accelerator: 3224
  Name: 7494
  Accelerator Cores: 7945
  Effeciency (%): 1000
  Mflops/Watt: 3028
  Processor Technology: 13500
  Processor Speed (MHz): 13500
  OS Family: 13500
  Cores per Socket: 13500
  Processor Generation: 13500
  Previous Rank: 10447
  First Appearance: 13000
  First Rank: 13000
  E

,Rank,Site,Manufacturer,Computer,Country,Year,Processors,Nmax,Nhalf,Processor Family,...,Efficiency (%),Power Source,HPCG [TFlop/s],Power Effeciency [GFlops/Watts],Site ID,System ID,Energy Efficiency [GFlops/Watts],RMax (Flop/s),RPeak (Flop/s),Power (W)
0,1,Los Alamos National Laboratory,Thinking Machines Corporation,CM-5/1024,United States,1993,1024.0,52224.0,24064.0,Sparc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.970000e+01,1.310000e+02,NaN
1,2,Minnesota Supercomputer Center,Thinking Machines Corporation,CM-5/544,United States,1993,544.0,36864.0,16384.0,Sparc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.040000e+01,6.963000e+01,NaN
2,3,NCSA,Thinking Machines Corporation,CM-5/512,United States,1993,512.0,36864.0,16384.0,Sparc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.040000e+01,6.554000e+01,NaN
3,4,National Security Agency,Thinking Machines Corporation,CM-5/512,United States,1993,512.0,36864.0,16384.0,Sparc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.040000e+01,6.554000e+01,NaN
4,5,NEC,NEC,SX-3/44R,Japan,1990,4.0,6400.0,830.0,NEC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.320000e+01,2.560000e+01,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31995,496,Service Provider T,Lenovo,"Lenovo HR650x, Xeon Gold 6133 20C 2.5GHz, 25G ...",China,2019,56000.0,5184768.0,NaN,NaN,...,NaN,NaN,NaN,NaN,50329.0,179654.0,NaN,2.317950e+15,4.480000e+15,NaN
31996,497,Energy Company B,Inspur,"Inspur TS10000 HPC Server, Xeon Gold 6226R 16C...",China,2020,49920.0,1337000.0,668500.0,NaN,...,NaN,NaN,NaN,NaN,50842.0,179869.0,NaN,2.316000e+15,4.412100e+15,NaN
31997,498,Service Provider T,Lenovo,"ThinkSystem SR590, Xeon Gold 5218 16C 2.3GHz, ...",China,2020,108800.0,8079744.0,NaN,NaN,...,NaN,NaN,NaN,NaN,50329.0,179818.0,NaN,2.314210e+15,4.003840e+15,NaN
31998,499,Vienna Scientific Cluster,MEGWARE,"MEGWARE SLIDESX, AMD EPYC 7713 64C 2GHz, Infin...",Austria,2022,95232.0,6451200.0,NaN,NaN,...,NaN,Submitted,NaN,NaN,50345.0,180056.0,4.481202,2.312300e+15,3.047424e+15,516000.0


In [110]:
import pandas as pd
import os

def count_nans(df):
    """Returns the total number of NaN values in the DataFrame."""
    return df.isna().sum().sum()

# Load the original file
input_csv = os.path.join(os.getcwd(), "merged_top500_nans.csv")  # Update with your file path
df_original = pd.read_csv(input_csv, low_memory=False)

# Count NaNs before cleaning
nans_before = count_nans(df_original)
print(f"🔴 NaN Count BEFORE Cleaning: {nans_before}")

# Apply the cleaning function
df_cleaned = combine_columns(df_original.copy())  # Use the function from the previous script

# Count NaNs after cleaning
nans_after = count_nans(df_cleaned)
print(f"🟢 NaN Count AFTER Cleaning: {nans_after}")

# Save the cleaned file
output_csv = os.path.join(os.getcwd(), "merged_top500_clean.csv")
df_cleaned.to_csv(output_csv, index=False)

# Print reduction in NaNs
nans_removed = nans_before - nans_after
print(f"✅ NaN Reduction: {nans_removed} cells removed")


🔴 NaN Count BEFORE Cleaning: 1001831
🟢 NaN Count AFTER Cleaning: 713831
✅ NaN Reduction: 288000 cells removed
